In [ ]:
import pandas as pd
import numpy as np
import os 
import glob
import pydicom
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
pd.set_option('display.max_columns', None)

In [ ]:
# Reference to https://www.kaggle.com/seraphwedd18/pe-detection-with-keras-model-creation#Check-Targets-and-Input-Image
import vtk
from vtk.util import numpy_support
import cv2
reader = vtk.vtkDICOMImageReader()

def read_dicom(path):
    reader.SetFileName(path)
    reader.Update()
    _extent = reader.GetDataExtent()
    # ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1]

    imageData = reader.GetOutput()
    pointData = imageData.GetPointData()
    arrayData = pointData.GetArray(0)
    ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
    ArrayDicom = np.rot90(ArrayDicom)  # rotate then equal to pydicom.read_file
    return ArrayDicom

In [ ]:
df = pd.read_csv("../input/rsna-ped-dicom-metadata/train_with_dcm_metadata.csv")

In [ ]:
df

In [ ]:
df[df["StudyInstanceUID"]=="6d306dd47c57"]["dcm_InstanceNumber"].nunique()

In [ ]:
df[df["StudyInstanceUID"]=="6d306dd47c57"].sort_values(by="dcm_InstanceNumber")

In [ ]:
import math
import random
def show_exam_dcm(df, exam_id=None, window_center=None, window_width=None):
    if exam_id is None:
        exam_id = random.sample( list(df["StudyInstanceUID"].unique()), 1)[0]
    
    df_exam = df[df["StudyInstanceUID"]==exam_id].sort_values(by="dcm_InstanceNumber",ascending=False).reset_index(drop=True)
 
    dcm_dir = "../input/rsna-str-pulmonary-embolism-detection/train"
    cols_window = ["dcm_WindowCenter_0", "dcm_WindowWidth_0", "dcm_RescaleIntercept", "dcm_RescaleSlope"]
    dcm_num = df_exam.shape[0]
    cols = 6
    rows = math.ceil(dcm_num/cols)

    fig,ax = plt.subplots(rows,cols,figsize=[18,3*rows])
    for i in range( min( rows*cols, dcm_num )):
        StudyInstanceUID, SeriesInstanceUID, SOPInstanceUID, dcm_InstanceNumber = \
            df_exam.loc[i, ["StudyInstanceUID", "SeriesInstanceUID", "SOPInstanceUID", "dcm_InstanceNumber"]].values
        
        default_center, default_width, intercept, slope = df_exam.loc[i, cols_window].values
    
        path = f"{dcm_dir}/{StudyInstanceUID}/{SeriesInstanceUID}/{SOPInstanceUID}.dcm"
        
        dcm = read_dicom(path).astype(np.float32)
#         dcm = pydicom.read_file(path).pixel_array.astype(np.float32)
        
#         dcm =  dcm * slope + intercept
        
        if window_center=="default":
            window_center = default_center
        if window_width=="default":
            window_width = default_width
        if  window_center is not None and window_width is not None:
            window_min = window_center - window_width//2
            window_max = window_center + window_width//2
            dcm[dcm < window_min] = window_min
            dcm[dcm > window_max] = window_max
        
        img = ( (dcm - np.min(dcm)) / (np.max(dcm) - np.min(dcm)) )*255
      
        ax[int(i/cols),int(i % cols)].set_title(f'{exam_id} {dcm_InstanceNumber}')
        ax[int(i/cols),int(i % cols)].imshow(img  ,cmap='gray')
        ax[int(i/cols),int(i % cols)].axis('off')
        
#     plt.tight_layout()
    plt.show()
    

In [ ]:
show_exam_dcm(df, "6d306dd47c57")

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df)

In [ ]:
show_exam_dcm(df, window_center=50, window_width=450)

In [ ]:
show_exam_dcm(df, window_center=50, window_width=450)

In [ ]:
show_exam_dcm(df, window_center=50, window_width=450)

In [ ]:
show_exam_dcm(df, window_center=50, window_width=450)

In [ ]:
show_exam_dcm(df, window_center=50, window_width=450)

In [ ]:
show_exam_dcm(df, window_center=50, window_width=450)

In [ ]:
show_exam_dcm(df, window_center="default", window_width="default")

In [ ]:
show_exam_dcm(df, window_center="default", window_width="default")

In [ ]:
show_exam_dcm(df, window_center="default", window_width="default")

In [ ]:
show_exam_dcm(df, window_center="default", window_width="default")

In [ ]:
show_exam_dcm(df, window_center="default", window_width="default")